<a href="https://colab.research.google.com/github/gonzalez-kevin/PrelimRevenueEstimates/blob/main/Revenue%20Estimates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install XlsxWriter
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 3.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import duckdb

# Code for in Current Month

In [9]:
curr_month = """
WITH data AS (
    SELECT
        "Jobsite Code" AS job_number,
        CAST(DATE_TRUNC('month', CURRENT_DATE) AS DATE) AS work_date,
        COALESCE("Description", '') || ' ' || COALESCE("Jobsite Code", '') || '-' || COALESCE("Line No.", '') AS description,
        CASE
            WHEN "Price Type" = 'Per Month' THEN 'Monthly'
            ELSE 'Onetime'
        END AS UOM,
        CAST(REPLACE(CAST("Quantity" AS VARCHAR), ',', '') AS FLOAT) AS quantity,
        ROUND(CAST(REPLACE(CAST("Unit Price Excl. Tax" AS VARCHAR), ',', '') AS FLOAT), 2) AS unit_price,
        CAST("Start Date" AS DATE) AS start_date,
        CAST("Actual Stop Date" AS DATE) AS stop_date,
        "Invoice Detail ID Description" AS gl_description
    FROM
        "RPM Document Lines.csv"
    WHERE
        CAST(REPLACE(CAST("Quantity Shipped" AS VARCHAR), ',', '') AS FLOAT) > 0
        AND (
            "Actual Stop Date" IS NULL
            OR CAST("Actual Stop Date" AS DATE) >= DATE_TRUNC('month', CURRENT_DATE)
        )
)

SELECT
    job_number,
    CAST(work_date AS VARCHAR) AS work_date,
    description,
    UOM,
    ROUND(
        CASE
            WHEN UOM = 'Onetime' THEN ROUND(unit_price * quantity, 2)
            ELSE
                ROUND(
                CASE
                    WHEN start_date IS NOT NULL AND stop_date IS NOT NULL
                         AND DATE_TRUNC('month', start_date) = DATE_TRUNC('month', CURRENT_DATE)
                         AND DATE_TRUNC('month', stop_date) = DATE_TRUNC('month', CURRENT_DATE) THEN
                        (unit_price * quantity /
                        EXTRACT(DAY FROM DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')) *
                        (stop_date - start_date + 1)

                    WHEN start_date IS NOT NULL
                         AND DATE_TRUNC('month', start_date) = DATE_TRUNC('month', CURRENT_DATE) THEN
                        (unit_price * quantity /
                        EXTRACT(DAY FROM DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')) *
                        ((DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')::DATE - start_date + 1)

                    WHEN stop_date IS NOT NULL
                         AND DATE_TRUNC('month', stop_date) = DATE_TRUNC('month', CURRENT_DATE) THEN
                        (unit_price * quantity /
                        EXTRACT(DAY FROM DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')) *
                        (stop_date - DATE_TRUNC('month', CURRENT_DATE)::DATE + 1)

                    ELSE unit_price * quantity
                END, 2)
        END, 2) AS rate,
    1 AS quantity,
    ROUND(
        CASE
            WHEN UOM = 'Onetime' THEN ROUND(unit_price * quantity, 2)
            ELSE
                ROUND(
                CASE
                    WHEN start_date IS NOT NULL AND stop_date IS NOT NULL
                         AND DATE_TRUNC('month', start_date) = DATE_TRUNC('month', CURRENT_DATE)
                         AND DATE_TRUNC('month', stop_date) = DATE_TRUNC('month', CURRENT_DATE) THEN
                        (unit_price * quantity /
                        EXTRACT(DAY FROM DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')) *
                        (stop_date - start_date + 1)

                    WHEN start_date IS NOT NULL
                         AND DATE_TRUNC('month', start_date) = DATE_TRUNC('month', CURRENT_DATE) THEN
                        (unit_price * quantity /
                        EXTRACT(DAY FROM DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')) *
                        ((DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')::DATE - start_date + 1)

                    WHEN stop_date IS NOT NULL
                         AND DATE_TRUNC('month', stop_date) = DATE_TRUNC('month', CURRENT_DATE) THEN
                        (unit_price * quantity /
                        EXTRACT(DAY FROM DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')) *
                        (stop_date - DATE_TRUNC('month', CURRENT_DATE)::DATE + 1)

                    ELSE unit_price * quantity
                END, 2)
        END, 2) AS extension,
    CAST(start_date AS VARCHAR) AS start_date,
    COALESCE(CAST(stop_date AS VARCHAR), '') AS stop_date,
    gl_description
FROM
    data
WHERE
    (UOM = 'Monthly'
    AND start_date < DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month')
    AND (stop_date IS NULL OR stop_date >= DATE_TRUNC('month', CURRENT_DATE)))
    OR
    (UOM = 'Onetime'
    AND start_date >= DATE_TRUNC('month', CURRENT_DATE)
    AND start_date < DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month'))
ORDER BY
    job_number;
"""

df1 = duckdb.query(curr_month).df()  # Convert the result to a Pandas DataFrame



In [8]:
# Update values in dataframe and create a pivot table
# Ensure 'rate' and 'extension' columns are of type float and round to two decimal places
df1['gl_description'].replace('ECAM Damage Waiver', 'ECAM Other Rev & Reimbursement', inplace=True)
df1['rate'] = df1['rate'].astype(float).round(2)
df1['extension'] = df1['extension'].astype(float).round(2)

# Convert date columns to datetime format
date_columns = ['work_date', 'start_date', 'stop_date']
for col in date_columns:
    df1[col] = pd.to_datetime(df1[col], errors='coerce')

# Write dataframes to an excel file with short date format
with pd.ExcelWriter('Revenue Estimates.xlsx', engine='xlsxwriter') as writer:
    df1.to_excel(writer, sheet_name='Data', index=False)

    # Access the workbook and worksheet objects
    workbook  = writer.book
    worksheet = writer.sheets['Data']

    # Define date format for the specified columns
    date_format = workbook.add_format({'num_format': 'mm/dd/yyyy'})

    # Apply the date format to the date columns
    for idx, col in enumerate(df1.columns):
        if col in date_columns:
            worksheet.set_column(idx, idx, 15, date_format)  # Adjust the width if necessary


<ipython-input-8-dcaf380f92bd>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['gl_description'].replace('ECAM Damage Waiver', 'ECAM Other Rev & Reimbursement', inplace=True)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Code to run if in new month

In [10]:
prev_month = """
WITH data AS (
    SELECT
        "Jobsite Code" AS job_number,
        CAST(DATE_TRUNC('month', CURRENT_DATE) AS DATE) AS work_date,
        COALESCE("Description", '') || ' ' || COALESCE("Jobsite Code", '') || '-' || COALESCE("Line No.", '') AS description,
        CASE
            WHEN "Price Type" = 'Per Month' THEN 'Monthly'
            ELSE 'Onetime'
        END AS UOM,
        CAST(REPLACE(CAST("Quantity" AS VARCHAR), ',', '') AS FLOAT) AS quantity,
        ROUND(CAST(REPLACE(CAST("Unit Price Excl. Tax" AS VARCHAR), ',', '') AS FLOAT), 2) AS unit_price,
        CAST("Start Date" AS DATE) AS start_date,
        CAST("Actual Stop Date" AS DATE) AS stop_date,
        "Invoice Detail ID Description" AS gl_description
    FROM
        "RPM Document Lines.csv"
    WHERE
        CAST(REPLACE(CAST("Quantity Shipped" AS VARCHAR), ',', '') AS FLOAT) > 0
        AND (
            "Actual Stop Date" IS NULL
            OR CAST("Actual Stop Date" AS DATE) >= DATE_TRUNC('month', CURRENT_DATE)
        )
)

SELECT
    job_number,
    CAST(work_date AS VARCHAR) AS work_date,
    description,
    UOM,
    ROUND(
        CASE
            WHEN UOM = 'Onetime' THEN ROUND(unit_price * quantity, 2)
            ELSE
                ROUND(
                CASE
                    WHEN start_date IS NOT NULL AND stop_date IS NOT NULL
                         AND DATE_TRUNC('month', start_date) = DATE_TRUNC('month', CURRENT_DATE)
                         AND DATE_TRUNC('month', stop_date) = DATE_TRUNC('month', CURRENT_DATE) THEN
                        (unit_price * quantity /
                        EXTRACT(DAY FROM DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')) *
                        (stop_date - start_date + 1)

                    WHEN start_date IS NOT NULL
                         AND DATE_TRUNC('month', start_date) = DATE_TRUNC('month', CURRENT_DATE) THEN
                        (unit_price * quantity /
                        EXTRACT(DAY FROM DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')) *
                        ((DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')::DATE - start_date + 1)

                    WHEN stop_date IS NOT NULL
                         AND DATE_TRUNC('month', stop_date) = DATE_TRUNC('month', CURRENT_DATE) THEN
                        (unit_price * quantity /
                        EXTRACT(DAY FROM DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')) *
                        (stop_date - DATE_TRUNC('month', CURRENT_DATE)::DATE + 1)

                    ELSE unit_price * quantity
                END, 2)
        END, 2) AS rate,
    1 AS quantity,
    ROUND(
        CASE
            WHEN UOM = 'Onetime' THEN ROUND(unit_price * quantity, 2)
            ELSE
                ROUND(
                CASE
                    WHEN start_date IS NOT NULL AND stop_date IS NOT NULL
                         AND DATE_TRUNC('month', start_date) = DATE_TRUNC('month', CURRENT_DATE)
                         AND DATE_TRUNC('month', stop_date) = DATE_TRUNC('month', CURRENT_DATE) THEN
                        (unit_price * quantity /
                        EXTRACT(DAY FROM DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')) *
                        (stop_date - start_date + 1)

                    WHEN start_date IS NOT NULL
                         AND DATE_TRUNC('month', start_date) = DATE_TRUNC('month', CURRENT_DATE) THEN
                        (unit_price * quantity /
                        EXTRACT(DAY FROM DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')) *
                        ((DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')::DATE - start_date + 1)

                    WHEN stop_date IS NOT NULL
                         AND DATE_TRUNC('month', stop_date) = DATE_TRUNC('month', CURRENT_DATE) THEN
                        (unit_price * quantity /
                        EXTRACT(DAY FROM DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month') - INTERVAL '1 day')) *
                        (stop_date - DATE_TRUNC('month', CURRENT_DATE)::DATE + 1)

                    ELSE unit_price * quantity
                END, 2)
        END, 2) AS extension,
    CAST(start_date AS VARCHAR) AS start_date,
    COALESCE(CAST(stop_date AS VARCHAR), '') AS stop_date,
    gl_description
FROM
    data
WHERE
    (UOM = 'Monthly'
    AND start_date < DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month')
    AND (stop_date IS NULL OR stop_date >= DATE_TRUNC('month', CURRENT_DATE)))
    OR
    (UOM = 'Onetime'
    AND start_date >= DATE_TRUNC('month', CURRENT_DATE)
    AND start_date < DATE_TRUNC('month', CURRENT_DATE + INTERVAL '1 month'))
ORDER BY
    job_number;
"""

df2 = duckdb.query(prev_month).df()  # Convert the result to a Pandas DataFrame



In [11]:
# Update values in dataframe and create a pivot table
# Ensure 'rate' and 'extension' columns are of type float and round to two decimal places
df2['gl_description'].replace('ECAM Damage Waiver', 'ECAM Other Rev & Reimbursement', inplace=True)
df2['rate'] = df2['rate'].astype(float).round(2)
df2['extension'] = df2['extension'].astype(float).round(2)

# Convert date columns to datetime format
date_columns = ['work_date', 'start_date', 'stop_date']
for col in date_columns:
    df2[col] = pd.to_datetime(df2[col], errors='coerce')

# Write dataframes to an excel file with short date format
with pd.ExcelWriter('PrvMo Revenue Estimates.xlsx', engine='xlsxwriter') as writer:
    df2.to_excel(writer, sheet_name='Data', index=False)

    # Access the workbook and worksheet objects
    workbook  = writer.book
    worksheet = writer.sheets['Data']

    # Define date format for the specified columns
    date_format = workbook.add_format({'num_format': 'mm/dd/yyyy'})

    # Apply the date format to the date columns
    for idx, col in enumerate(df2.columns):
        if col in date_columns:
            worksheet.set_column(idx, idx, 15, date_format)  # Adjust the width if necessary


<ipython-input-11-9d2a3d874a9a>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['gl_description'].replace('ECAM Damage Waiver', 'ECAM Other Rev & Reimbursement', inplace=True)
